<a href="https://colab.research.google.com/github/QZKasoma-Luo/CUDA-ImageClassifier/blob/main/save_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from numba import jit, cuda
import math
import time
import random


data = pd.read_csv('train.csv')

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [ ]:
from __future__ import division


# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

# Host code

# Initialize the data arrays
def matmul_host(A: np.ndarray, B: np.ndarray, A_shape: tuple, B_shape: tuple):

  # Copy the arrays to the device
  A_global_mem = cuda.to_device(A)
  B_global_mem = cuda.to_device(B)

  # Allocate memory on the device for the result
  C_global_mem = cuda.device_array((A_shape[0], B_shape[1]))

  # Configure the blocks
  threadsperblock = (32, 32)
  blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
  blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
  blockspergrid = (blockspergrid_x, blockspergrid_y)

  # Start the kernel
  matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

  # Copy the result back to the host
  C = C_global_mem.copy_to_host()
  return C

A = np.array([[3,4,7],[1,2,3]])
B = np.array([[2],[1],[1]])

C = matmul_host(A, B, A.shape, B.shape)
print(C)
print(type(C))


[[17.]
 [ 7.]]
<class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [ ]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = matmul_host(W1, X, W1.shape, X.shape) + b1
    A1 = ReLU(Z1)
    Z2 = matmul_host(W2, A1, W2.shape, A1.shape) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * matmul_host(dZ2, A1.T, dZ2.shape, A1.T.shape)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = matmul_host(W2.T, dZ2, W2.T.shape, dZ2.shape) * ReLU_deriv(Z1)
    dW1 = 1 / m * matmul_host(dZ1, X.T, dZ1.shape, X.T.shape)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # if i % 10 == 0:
        #     print("Iteration: ", i)
        #     predictions = get_predictions(A2)
        #     print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
time_s_gpu = time.time()
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)
time_e_gpu = time.time()
print("Time gggggggpu taken: ", time_e_gpu - time_s_gpu)



Iteration:  0
[4 1 1 ... 1 9 8] [1 2 2 ... 7 2 8]
0.09885365853658537
Iteration:  10
[4 1 2 ... 1 9 8] [1 2 2 ... 7 2 8]
0.20834146341463414
Iteration:  20
[1 1 2 ... 1 9 8] [1 2 2 ... 7 2 8]
0.29665853658536584
Iteration:  30
[1 1 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.3635853658536585
Iteration:  40
[1 8 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.4277560975609756
Iteration:  50
[1 8 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.49034146341463414
Iteration:  60
[1 8 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.5425609756097561
Iteration:  70
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.5883170731707317
Iteration:  80
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.6216829268292683
Iteration:  90
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.6476829268292683
Iteration:  100
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.6694146341463415
Iteration:  110
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.6865853658536586
Iteration:  120
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.7016829268292682
Iteration:  130
[1 2 2 ... 1 2 8] [1 2 2 ... 7 2 8]
0.7140731707317073
Iteration:  1